In [256]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor

In [257]:
r = requests.get(url).json()
total_pages = int(r['totalPages'])
total_pages
df3 = pd.DataFrame()
def fetch_auction_house(page_number:int) -> pd.DataFrame:
        response = requests.get(f'{url}?page={page_number}')
        auctions_data = response.json()['auctions']
        df = pd.DataFrame(auctions_data)
        df.drop(columns=['auctioneer','profile_id','coop', 'extra','item_lore','tier','item_bytes','claimed_bidders','item_uuid','bids'],inplace=True)
        return df

def clean_dataframe(df:pd.DataFrame) -> pd.DataFrame:
        df['start'] = pd.to_datetime(df['start'], unit='ms')
        df['end'] = pd.to_datetime(df['end'], unit='ms')
        df['last_updated'] = pd.to_datetime(df['last_updated'], unit='ms')
        return df



for i in range(total_pages):
   df3 = pd.concat( [df3, fetch_auction_house(i)])

df = clean_dataframe(df3)

df

,uuid,start,end,item_name,category,starting_bid,claimed,highest_bid_amount,last_updated,bin
0,75806f3ae565468e8bbd07d7893a4d03,2024-07-14 03:49:32.163,2024-07-14 09:49:32.163,Aurora Leggings,armor,8900000,False,0,2024-07-14 03:49:32.163,True
1,dd3c4bf6132e4be1b43d18240e4920f4,2024-07-14 03:49:32.009,2024-07-14 09:49:32.009,Healing Ring,accessories,300000,False,0,2024-07-14 03:49:32.009,True
2,d385cc91040940398b6c3305df1fd61e,2024-07-13 15:52:25.769,2024-07-14 03:52:25.769,Aspect of the Dragons,weapon,10,False,1035683,2024-07-14 03:49:31.735,False
3,2123f53cb37c409d981e6759fb22813c,2024-07-14 03:49:31.260,2024-07-14 15:49:31.260,Clean Skeleton Master Boots,armor,20000,False,0,2024-07-14 03:49:31.260,True
4,16920231de58485b869cd8ea17adf3a9,2024-07-14 03:49:31.235,2024-07-14 09:49:31.235,Fish Affinity Talisman,accessories,60000,False,0,2024-07-14 03:49:31.235,True
...,...,...,...,...,...,...,...,...,...,...
730,16da74bf47f14517baa303c43d7bb757,2024-06-30 03:03:07.235,2024-07-14 05:03:07.235,Judgement Core,misc,406000000,False,0,2024-06-30 03:03:07.235,True
731,b554ea6158f1409fb742e6fd51c5eaa2,2024-06-30 02:38:49.438,2024-07-14 04:38:49.438,Plushie Reindeer Skin,misc,499999999,False,0,2024-06-30 02:38:49.438,True
732,4755bb995d68471d83f9de70d6a6f58d,2024-06-30 02:25:28.835,2024-07-14 04:25:28.835,Crochet Tiger Plushie,misc,26000000,False,0,2024-06-30 02:25:28.835,True
733,a7b5a37a604e47cc933d110a81c12c16,2024-06-30 02:25:06.798,2024-07-14 04:25:06.798,Barry Century Cake,misc,50000000,False,0,2024-06-30 02:25:06.798,True


In [258]:
# Using threading improved avg runtime from 25 - 30seconds to 3 - 4 (Thanks ChatGPT for the help)

r = requests.get(url).json()
total_pages = int(r['totalPages'])


# For loops through the total_number of pages on the market place, requesting JSON for each page
# Each page contains a max of 1000 Items
def fetch_auction_house(page_number: int) -> pd.DataFrame:
    response = requests.get(f'{url}?page={page_number}')
    auctions_data = response.json()['auctions']
    df = pd.DataFrame(auctions_data)
    return df
# Clean DataFrame
# Remove unnescessary code
# Change all timestamps from UNIX to yyyy/mm/dd hour/min/sec/ms format
def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    df.drop(columns=['auctioneer', 'profile_id', 'coop', 'extra', 'item_lore', 'tier', 'item_bytes', 'claimed_bidders', 'item_uuid', 'bids'], inplace=True)
    df['start'] = pd.to_datetime(df['start'], unit='ms')
    df['end'] = pd.to_datetime(df['end'], unit='ms')
    df['last_updated'] = pd.to_datetime(df['last_updated'], unit='ms')
    return df


#ChatGPT helped me with this one, ill need someone to explain this to me
# Use ThreadPoolExecutor to fetch pages concurrently
def fetch_all_pages_concurrently(total_pages: int) -> pd.DataFrame:
    with ThreadPoolExecutor(max_workers=15) as executor:
        futures = [executor.submit(fetch_auction_house, i) for i in range(total_pages)]
        result = pd.concat([future.result() for future in futures], ignore_index=True)
    return result

df3 = fetch_all_pages_concurrently(total_pages)
df = clean_dataframe(df3)

df

,uuid,start,end,item_name,category,starting_bid,claimed,highest_bid_amount,last_updated,bin
0,9b08c4698c2e418ebb6ea30308d0c19a,2024-07-14 03:50:31.764,2024-07-16 03:50:31.764,Attribute Shard,misc,3790000,False,0,2024-07-14 03:50:31.764,True
1,11425a376e2d45998f50f2be0e08c74c,2024-07-14 03:50:31.699,2024-07-28 03:50:31.699,Zooming Pythagorean Potato Hoe,misc,4000000,False,0,2024-07-14 03:50:31.699,True
2,bf8381bdd7f6457abb5ea0f6f0a52b09,2024-07-14 03:50:31.484,2024-07-14 09:50:31.484,Radiant Power Orb,misc,55000,False,0,2024-07-14 03:50:31.484,True
3,cdb966c63faa45ab8a94e48bd341f346,2024-07-14 03:50:31.380,2024-07-14 09:50:31.380,God Potion,misc,2000000,False,0,2024-07-14 03:50:31.380,True
4,34d7326717774ec19ee773daef2923c6,2024-07-14 03:50:30.967,2024-07-16 03:50:30.967,[Lvl 1] Silverfish,misc,2290000,False,0,2024-07-14 03:50:30.967,True
...,...,...,...,...,...,...,...,...,...,...
58730,16da74bf47f14517baa303c43d7bb757,2024-06-30 03:03:07.235,2024-07-14 05:03:07.235,Judgement Core,misc,406000000,False,0,2024-06-30 03:03:07.235,True
58731,b554ea6158f1409fb742e6fd51c5eaa2,2024-06-30 02:38:49.438,2024-07-14 04:38:49.438,Plushie Reindeer Skin,misc,499999999,False,0,2024-06-30 02:38:49.438,True
58732,4755bb995d68471d83f9de70d6a6f58d,2024-06-30 02:25:28.835,2024-07-14 04:25:28.835,Crochet Tiger Plushie,misc,26000000,False,0,2024-06-30 02:25:28.835,True
58733,a7b5a37a604e47cc933d110a81c12c16,2024-06-30 02:25:06.798,2024-07-14 04:25:06.798,Barry Century Cake,misc,50000000,False,0,2024-06-30 02:25:06.798,True


In [259]:
def find_item(item_name:str, specified_price:int) -> pd.DataFrame:
    filtered_df = df[df['item_name'] == item_name]
    filtered_df = filtered_df.sort_values(by=['starting_bid'])
    filtered_df = filtered_df.query(f'starting_bid < {specified_price} and bin == True and claimed == False')
    filtered_df.loc[:,'uuid'] = '/viewauction ' + filtered_df['uuid'].astype(str)
    if filtered_df.empty:
        return "Item not with specified Parameters"
    return filtered_df

find_item('Portal Dye', 79000000)

,uuid,start,end,item_name,category,starting_bid,claimed,highest_bid_amount,last_updated,bin
272,/viewauction 02c5bb733c7e4e36866bfd3640647f47,2024-07-14 03:48:48.189,2024-07-15 03:48:48.189,Portal Dye,misc,77899000,False,0,2024-07-14 03:48:48.189,True
126,/viewauction 7ff83ded3b8f418892ebb074d1d13e41,2024-07-14 03:49:54.571,2024-07-14 09:49:54.571,Portal Dye,misc,78000000,False,0,2024-07-14 03:49:54.571,True
